In [2]:
import numpy as np

In [3]:
import astropy.units as u

In [4]:
from astropy.time import Time

In [5]:
from astropy.table import Table

In [6]:
from sbpy.data import Ephem, Phys

In [7]:
from sbpy.activity import Haser, LTE, photo_timescale, einstein_coeff, intensity_conversion, beta_factor, total_number_nocd

Production Rate without Photolysis
--------------------------------

`sbpy.spectroscopy` provides several models to calculate production rates in comets. One of the models followed by this module is based on the following paper:

Drahus et al. September 2012. The Sources of HCN and CH3OH and the
Rotational Temperature in Comet 103P/Hartley 2 from Time-resolved
Millimeter Spectroscopy. The Astrophysical Journal, Volume 756,
Issue 1.

This model does not include photolysis effects. For more information on the parameters that are optional or needed for the module look at the documentation. This extemely simplified model is useful to calculate first guesses for more computationally intensive Non-LTE models or for LTE Haser models that include photolysis effects.

The following examples test this simple version of the production rate calculation and they are based on data from the existing literature listed above.

In [8]:
hcn = Table.read('data/HCN.csv', format="ascii.csv")

temp_estimate = 47. * u.K
target = '103P'
vgas = 0.8 * u.km / u.s
aper = 30 * u.m  # The aperture for telescope used (Drahus et al. 2012)
b = 1.13  # Value taken from (Drahus et al. 2012)
mol_tag = 27001
transition_freq = (265.886434 * u.GHz).to('MHz')
q_found_hcn = []
mol_data = Phys.from_jplspec(temp_estimate, transition_freq, mol_tag)
intl = intensity_conversion(mol_data)
mol_data.add_column([intl.value] * intl.unit,
                    name='Integrated line intensity at desired temp')
au = einstein_coeff(mol_data)
mol_data.add_column([au.value] * au.unit, name='eincoeff')

for i in range(0, 28):

    time = Time(hcn['Time'][i], format='iso')
    integrated_flux = hcn['T_B'][i] * u.K * u.km / u.s
    ephemobj = Ephem.from_horizons(target, epochs=time.jd, id_type='id')

    lte = LTE()

    q = lte.from_Drahus(integrated_flux, mol_data, ephemobj, vgas, aper, b=b)

    q = np.log10(q.value)

    q_found_hcn.append(q)

print(np.round(q_found_hcn,3))

[24.755 24.953 25.02  25.062 25.206 25.182 25.11  25.025 25.116 25.222
 25.176 25.274 25.213 24.989 25.152 25.199 25.148 25.025 24.901 24.943
 24.83  24.861 24.843 24.788 24.679 24.725 24.968 25.049]


In [9]:
# calculate error from data
q_pred_hcn = list(hcn['log(Q)']) 
err_hcn = abs((np.array(q_pred_hcn) - (np.array(q_found_hcn))) / np.array(q_pred_hcn) * 100)
print(np.round(err_hcn,3))

[0.234 0.231 0.223 0.223 0.22  0.216 0.216 0.186 0.184 0.182 0.18  0.178
 0.175 0.171 0.142 0.138 0.138 0.135 0.133 0.133 0.13  0.1   0.096 0.093
 0.091 0.088 0.085 0.082]


In [10]:
ch3oh = Table.read('data/CH3OH.csv', format="ascii.csv")
temp_estimate = 47. * u.K
target = '103P'
vgas = 0.8 * u.km / u.s
aper = 30 * u.m  # The aperture for telescope used (Drahus et al. 2012)
b = 1.13  # Value taken from (Drahus et al. 2012)
mol_tag = 32003
transition_freq = (157.178987 * u.GHz).to('MHz')
q_found_ch3oh = []
mol_data = Phys.from_jplspec(temp_estimate, transition_freq, mol_tag)
intl = intensity_conversion(mol_data)
mol_data.add_column([intl.value] * intl.unit,
                    name='Integrated line intensity at desired temp')
au = einstein_coeff(mol_data)
mol_data.add_column([au.value] * au.unit, name='eincoeff')

for i in range(0, 20):

    time = Time(ch3oh['Time'][i], format='iso')
    integrated_flux = ch3oh['T_B'][i] * u.K * u.km / u.s
    ephemobj = Ephem.from_horizons(target, epochs=time.jd, id_type='id')

    lte = LTE()

    q = lte.from_Drahus(integrated_flux, mol_data, ephemobj, vgas, aper, b=b)

    q = np.log10(q.value)

    q_found_ch3oh.append(q)   

print(np.round(q_found_ch3oh,3))

[26.445 26.573 26.563 26.593 26.597 26.491 26.48  26.553 26.504 26.482
 26.463 26.423 26.354 26.477 26.349 26.399 26.31  26.325 26.401 26.556]


In [11]:
# calculate error from data
q_pred_ch3oh = list(ch3oh['log(Q)']) 
err_ch3oh = abs((np.array(q_pred_ch3oh) - (np.array(q_found_ch3oh))) / np.array(q_pred_ch3oh) * 100)
print(np.round(err_ch3oh,3))

[0.334 0.34  0.338 0.331 0.331 0.329 0.302 0.3   0.302 0.291 0.293 0.291
 0.296 0.257 0.268 0.257 0.247 0.263 0.25  0.252]


Calculating Production Rate using `sbpy.activity.gas` Haser model
-------------------------------------------------------------

Another model included in the module is based off of the model in the following literature:

Haser 1957, Bulletin de la Societe Royale des Sciences de Liege 43, 740.
Newburn and Johnson 1978, Icarus 35, 360-368.

This model takes in an initial guess for the production rate, and uses the Haser module found in sbpy.activity.gas to find a ratio between the model model total number of molecules and the number of molecules calculated from the data to scale the model Q and output the new production rate from the result. This model does account for the effects of photolysis.

In [12]:

co = Table.read(('data/CO.csv'), format="ascii.csv")

Q_estimate = 2.8*10**(28) / u.s
transition_freq = (230.53799 * u.GHz).to('MHz')
aper = 10 * u.m
mol_tag = 28001
temp_estimate = 25. * u.K
vgas = 0.5 * u.km / u.s
target = 'C/2016 R2'
b = 0.74
mol_data = Phys.from_jplspec(temp_estimate, transition_freq, mol_tag)
intl = intensity_conversion(mol_data)
mol_data.add_column([intl.value] * intl.unit,
                    name='Integrated line intensity at desired temp')
au = einstein_coeff(mol_data)
mol_data.add_column([au.value] * au.unit, name='eincoeff')
mol_data.add_column([1.] * u.AU * u.AU * u.s, name='beta')
mol_data.add_column([1.], name='total_number_nocd')

q_found_co = []

parent = photo_timescale('CO') * vgas
coma = Haser(Q_estimate, vgas, parent)

for i in range(0, 5):

    time = Time(co['Time'][i], format='iso')
    integrated_flux = co['T_B'][i] * u.K * u.km / u.s
    ephemobj = Ephem.from_horizons(target, epochs=time.jd)
    beta = beta_factor(mol_data, ephemobj)
    mol_data['beta'] = beta
    tnum = total_number_nocd(integrated_flux, mol_data, aper, b)

    mol_data['total_number_nocd'] = tnum

    lte = LTE()

    Q = lte.from_Haser(coma, mol_data, aper=aper)

    q_found_co.append(np.log10(Q.value)[0])

print(np.round(q_found_co, 3))

[27.971 28.002 27.96  27.975 27.953]


In [13]:
# calculate percent difference between data and literature
q_pred_co = list(co['log(Q)']) 
err_co = abs((np.array(q_pred_co) - (np.array(q_found_co))) / np.array(q_pred_co) * 100)
print(np.round(err_co,3))

[2.347 2.305 2.453 2.4   2.476]
